In [1]:
import pandas as pd
import numpy as np

In [4]:
df=pd.read_csv('/content/drive/MyDrive/IMDB Dataset.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [6]:
from sklearn.preprocessing import LabelEncoder


In [7]:
label_encoder = LabelEncoder()
df['sentiment'] = label_encoder.fit_transform(df['sentiment'])

In [8]:
from sklearn.model_selection import train_test_split


In [9]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


In [10]:
max_words = 10000
max_length = 100

In [11]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [12]:
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(train_df['review'])


In [13]:
train_sequences = tokenizer.texts_to_sequences(train_df['review'])
test_sequences = tokenizer.texts_to_sequences(test_df['review'])

padded_train_sequences = pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')
padded_test_sequences = pad_sequences(test_sequences, maxlen=max_length, padding='post', truncating='post')

In [14]:
# Build the RNN model
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=16, input_length=max_length))
model.add(SimpleRNN(units=32))
model.add(Dense(units=1, activation='sigmoid'))

In [15]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [16]:
# Train the model
model.fit(padded_train_sequences, train_df['sentiment'], epochs=5, validation_split=0.2)


Epoch 1/5
1000/1000 [==============================] - 28s 26ms/step - loss: 0.6907 - accuracy: 0.5348 - val_loss: 0.6864 - val_accuracy: 0.5529
Epoch 2/5
1000/1000 [==============================] - 26s 26ms/step - loss: 0.6095 - accuracy: 0.6767 - val_loss: 0.6932 - val_accuracy: 0.5537
Epoch 3/5
1000/1000 [==============================] - 26s 26ms/step - loss: 0.4890 - accuracy: 0.7594 - val_loss: 0.8493 - val_accuracy: 0.5295
Epoch 4/5
1000/1000 [==============================] - 24s 24ms/step - loss: 0.3546 - accuracy: 0.8391 - val_loss: 0.9210 - val_accuracy: 0.5255
Epoch 5/5
1000/1000 [==============================] - 26s 26ms/step - loss: 0.2298 - accuracy: 0.9000 - val_loss: 1.1731 - val_accuracy: 0.5595


In [17]:
accuracy = model.evaluate(padded_test_sequences, test_df['sentiment'])[1]
print(f"Test Accuracy: {accuracy}")

313/313 [==============================] - 2s 6ms/step - loss: 1.1592 - accuracy: 0.5651
Test Accuracy: 0.5651000142097473


In [18]:
new_texts = ["This is a great movie!", "I didn't like the product."]
new_sequences = tokenizer.texts_to_sequences(new_texts)
padded_new_sequences = pad_sequences(new_sequences, maxlen=max_length, padding='post', truncating='post')

In [19]:
predictions = model.predict(padded_new_sequences)
print("Predictions:", predictions)

1/1 [==============================] - 0s 203ms/step
Predictions: [[0.1553256 ]
 [0.16154024]]
